## 环境的配置

1. 首先安装python3.6以上的版本
2. 安装pipenv 
`sudo pip install --user pipenv //这样将pipenv下载到~/local/bin文件中,然后将该文件添加到.profile文件中即可:vim .profile AND export PATH=$PATH:~/.local/bin`
3. 在工程目录下 `pipenv install`
![pipenv](../photo/pipenvinstall.png)

> To activate this project's virtualenv, run pipenv shell
4. 启动pipenv 安装项目包。 `pipenv install flask`

 - 卸载 `pipenv uninstall flask`
 - 退出 exit
 - 软件包依赖关系 pipenv graph

## 工具安装
1. pycharm
2. Xampp -> MySQL
3. Navicat 数据库可视化管理工具

## 环境配置
`pipenv --venv` 查看虚拟环境路径
> /home/tzone/.local/share/virtualenvs/fisher-M_H0keWs

## flask唯一url原则
兼容用户路径多加一个/,使用301重定向。
普通访问
![url1](../photo/url1.png)
加/访问
![url2](../photo/url2.png)
遵循唯一url原则。


## 路由的另一种方法和原理

更新代码自动重启服务器：`app.run(debug=True)`

**chrom浏览器有数据缓存**，如果访问**not found**可以清除浏览器缓存chrome://settings/clearBrowserData?search=%E6%B8%85%E9%99%A4%E6%B5%8F%E8%A7%88

#app.add_url_rule('/hello', view_func=hello) #另一种注册路由的方法

> 路由注册原理：

(查看源码 ctrl+b)
![decorator](../photo/decorator.png)



## app.run相关参数与flask配置文件

> host='0.0.0.0' #所有局域网IP均可访问
> port= #修改端口号

flask配置文件相对自由
创建一个config.py ，然后`app.config.from_object('config')`引入配置文件，`app.run(host=app.config['HOST'], debug=app.config['DEBUG'])`指定参数。

这里app.config是一个dict的字类，config中参数名称必须大写，如果小写会被忽略掉。

## if __name__= '__main__':

![main](../photo/main.png)

```python
if __name__ == '__main__':
    # 生产环境 nginx + uwsgi
    app.run(host=app.config['HOST'], debug=app.config['DEBUG'])

```
一般使用nginx接受web请求，然后使用uwssgi加载相关模块文件启动flask相关代码，所以生产环境下app.run不会执行的。

## 视图函数renturn response对象

视图函数返回值是一个response对象
```python
from flask import make_response

def hello():
    headers = {
        'content-type': 'text/plain',
        'location': 'https://www.baidu.com'
    }
    response = make_response('<html></html>', 301)
    response.headers = headers
    return response
```
如果要提供json格式的数据
在response headers：`content-type: 'application/json'`

**另一种相对简单返回方法**：
`return 'content',301,headers` #没有使用response对象(本质都是resposne)，这里逗号分割的参数，是一个元祖，如果要返回cookie也是调用response下的setcookie方法。


## 路由传参
1. `@app.route('/book/search/<q>/<page>')`

## 深度理解flask路由

> 1. 如何通过url访问到视图函数的？

url - endpoint - viewfunction

一个url会有一个endpoint还会有一个视图函数。

endpoint用来反向构建url。

`app.add_url_rule(‘url’,view_func=,endpoint=) `如果不传endpoint则会把视图函数的函数名作为endpoint的默认值。

### 调试
>  把DEBUG调试模式改为False,启动调试的话，将会把整体代码执行两次，会有一个restart过程。

ctrl B进入源码,在app/web/book.py app.route()源码处打一个断点，


## Request


from flask import request

request:中包含HTTP的请求信息，查询参数，POST参数，remote ip等信息

exampel：
http://**.com?q=1&?page=2

q = request.args['q']

page = request.args['page']

args是一个不可变字典。

通过request.args.to_dict()可以将args转换为一个可变字典。

# wtforms参数验证

> pip install wtforms

**验证层**
```python

from wtforms import Form, StringField, IntegerField
from wtforms.validators import Length, NumberRange, DataRequired


class SearchForm(Form):
    q = StringField(validators=[DataRequired(),Length(min=1, max=20)])
    page = IntegerField(validators=[NumberRange(min=1, max=10)], default=1)
```